In [1]:
!pip install biopython pyteomics


In [2]:
from bisect import bisect_left
from Bio import SeqIO
import re
from pyteomics import mgf, mass

def return_frag_mzs(peptide, z):
    mzValues = []
    digPat = r'[\+\-]\d+(\.\d+)?'  # Updated regex pattern
    digs = re.findall(digPat, peptide)
    pepFrags = re.split(digPat, peptide)
    modValues = {}
    seq = ''
    while len(digs) != 0:
        dig = digs.pop(0)
        frag = pepFrags.pop(0)
        seq += frag
        modValue = float(dig) / z  # Include the sign in conversion
        modValues[len(seq)] = modValue
    seq += pepFrags[0]
    # Calculate y-ions
    for i in range(1, len(seq)):
        mz = mass.fast_mass(sequence=seq[i:], ion_type='y', charge=z)
        mz += sum([modValues[pos] for pos in modValues if pos > i])
        mzValues.append(mz)
    # Calculate b-ions
    for i in range(1, len(seq)):
        mz = mass.fast_mass(sequence=seq[:i], ion_type='b', charge=z)
        mz += sum([modValues[pos] for pos in modValues if pos <= i])
        mzValues.append(mz)
    return mzValues

def approx_list(value, lst, tolerance=0.5):
    for idx, item in enumerate(lst):
        if abs(value - item) <= tolerance:
            return idx
    return -1

def clean_mgf_file(mgfFile, fasta, output_file, ions=False):
    spectra = mgf.read(mgfFile)
    fDict = {}
    longPep = ''
    
    # Parse the FASTA file and create a dictionary of protein sequences
    for record in SeqIO.parse(open(fasta, 'r'), 'fasta'):
        fDict[len(longPep)] = record.id
        longPep += str(record.seq) + '.'
    
    cleaned = []
    count = 0
    pepCount = 0
    
    # Updated regex pattern for modification removal
    mod_removal_pattern = r'[\+\-]\d+(\.\d+)?'
    
    for spec in spectra:
        count += 1
        
        if ions:
            mzValues = return_frag_mzs(spec['params']['seq'], 1)
            peaks = list(zip(spec['m/z array'], spec['intensity array']))
            for i in range(len(peaks)-1, -1, -1):
                if approx_list(peaks[i][0], mzValues) == -1:
                    peaks.pop(i)
            if len(peaks) == 0:
                continue
            peaks.sort(key=lambda x: x[0])
            spec['m/z array'], spec['intensity array'] = map(list, zip(*peaks))
        
        decoy = False
        if 'protein' in spec['params'] and 'DECOY' in spec['params']['protein']:
            decoy = True
        else:
            # Remove both positive and negative modifications
            seq = re.sub(mod_removal_pattern, '', spec['params']['seq'])
            listOfI = [m.start() for m in re.finditer(seq, longPep)]
            sorted_keys = sorted(fDict.keys())
            proteins = set()
            for i in listOfI:
                insertion_point = bisect_left(sorted_keys, i)
                if insertion_point == len(sorted_keys) or sorted_keys[insertion_point] != i:
                    insertion_point -= 1
                protein = fDict[sorted_keys[insertion_point]]
                proteins.add(protein)
            if len(proteins) == 0:
                proteins.add('protein_not_in_fasta_' + spec['params']['seq'])

        if decoy:
            proteins = ['DECOY_0_' + x for x in proteins]

        protein = str(len(proteins)) + '/' + '/'.join(sorted(proteins))
        if protein != '0/':
            spec['params']['protein'] = protein
            pepCount += 1
        
        cleaned.append(spec)
        
        if count % 1000 == 0:
            print(f"Processed {count} spectra; added protein info to {pepCount} spectra")
    
    # Write the cleaned spectra with protein information to a new MGF file
    mgf.write(cleaned, output_file)
    print(f"MGF file with added protein information has been saved to: {output_file}")

# Usage example (replace with your actual file paths)
mgf_file = r"F:\Libfile\Celegans_1_addseq"
fasta_file = r"F:\Libfile\uniprotkb_proteome_UP000001940_2025_01_23.fasta"
output_file = r"F:\Libfile\Celegans_1_addprot"

clean_mgf_file(mgf_file, fasta_file, output_file)


Processed 1000 spectra; added protein info to 1000 spectra
Processed 2000 spectra; added protein info to 2000 spectra
Processed 3000 spectra; added protein info to 3000 spectra
Processed 4000 spectra; added protein info to 4000 spectra
Processed 5000 spectra; added protein info to 5000 spectra
Processed 6000 spectra; added protein info to 6000 spectra
Processed 7000 spectra; added protein info to 7000 spectra
Processed 8000 spectra; added protein info to 8000 spectra
Processed 9000 spectra; added protein info to 9000 spectra
Processed 10000 spectra; added protein info to 10000 spectra
Processed 11000 spectra; added protein info to 11000 spectra
Processed 12000 spectra; added protein info to 12000 spectra
Processed 13000 spectra; added protein info to 13000 spectra
Processed 14000 spectra; added protein info to 14000 spectra
Processed 15000 spectra; added protein info to 15000 spectra
Processed 16000 spectra; added protein info to 16000 spectra
Processed 17000 spectra; added protein inf

In [7]:
from pyteomics import mgf

def add_decoy_to_title(input_mgf, output_mgf):
    spectra = mgf.read(input_mgf)
    modified_spectra = []
    count = 0
    decoy_count = 0

    for spec in spectra:
        count += 1
        params = spec['params']
        protein = params.get('PROTEIN', params.get('protein', ''))

        # Check if 'DECOY' is in the 'PROTEIN' parameter
        if 'DECOY' in protein:
            decoy_count += 1
            # Modify the 'TITLE' parameter
            if 'TITLE' in params:
                params['TITLE'] = 'DECOY_' + params['TITLE']
            elif 'title' in params:
                params['title'] = 'DECOY_' + params['title']
            else:
                # If 'TITLE' doesn't exist, create it
                params['TITLE'] = 'DECOY_Spectrum_' + str(count)
        else:
            # Ensure non-decoy spectra have a 'TITLE' parameter
            if 'TITLE' not in params and 'title' not in params:
                params['TITLE'] = 'Spectrum_' + str(count)

        modified_spectra.append(spec)

    # Write the modified spectra to a new MGF file
    mgf.write(modified_spectra, output_mgf)

    print(f"Total spectra processed: {count}")
    print(f"Total decoy spectra modified: {decoy_count}")
    print(f"Modified MGF file saved as: {output_mgf}")

# Usage example (replace with your actual file paths)
input_mgf = r"D:\DirectInfusion\Anti-body Monoclonal\RAW file for Lib\NIST_mAb_PTM_adddecoys.mgf"
output_mgf = r"D:\DirectInfusion\Anti-body Monoclonal\RAW file for Lib\NIST_mAb_PTM_adddecoysfixed.mgf"

add_decoy_to_title(input_mgf, output_mgf)


Total spectra processed: 27643
Total decoy spectra modified: 462
Modified MGF file saved as: D:\DirectInfusion\Anti-body Monoclonal\RAW file for Lib\NIST_mAb_PTM_adddecoysfixed.mgf
